## booklet question 9

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gammaln

In [2]:
def dirichlet_pdf(x, priors):
    x = np.array(x)
    x[x==0] += 1e-48
    x[x==1] -= 1e-48
    priors = np.array(priors)
    Z = np.sum(gammaln(priors)) - gammaln(np.sum(priors))
    loglikelihood = ((priors-1) * np.log(x)).sum(axis=1)
    posterior = loglikelihood - Z
    return np.exp(posterior)

In [3]:
A = np.array([6, 4, 3, 2, 1, 1, 1, 1, 1, 1e-48])
B = np.array([3, 3, 2, 2, 2, 2, 2, 2, 1, 1])
C = np.ones(10)
log_pdf_A = np.log(A / np.sum(A))
log_pdf_B = np.log(B / np.sum(B))
log_pdf_C = np.log(C / np.sum(C))

In [4]:
def one_hot(a, num_classes):
    return (np.eye(num_classes)[a.reshape(-1)])

In [5]:
# index starts at 0
x = np.array([5, 3, 9, 3, 8, 4, 7]) - 1
x = one_hot(x, 10)

In [6]:
pr_A = np.exp((x * log_pdf_A).sum())
pr_B = np.exp((x * log_pdf_B).sum())
pr_C = np.exp((x * log_pdf_C).sum())
print("prob A:", pr_A, "prob B:", pr_B, "prob C:", pr_C)

prob A: 1.4062500000000018e-08 prob B: 5.000000000000005e-08 prob C: 1.0000000000000029e-07


**unfinished**

## booklet question 26

In [7]:
X = np.array([1, 2])
Y = np.array([-1, 0, 5])
p_xy = np.array([[0.3, 0.3, 0.], [0.1, 0.2, 0.1]])

In [8]:
p_xy

array([[0.3, 0.3, 0. ],
       [0.1, 0.2, 0.1]])

### 1. Expectations

In [9]:
p_x = p_xy.sum(axis=1)
p_y = p_xy.sum(axis=0)
p_x_giveny = p_xy / p_y
p_y_givenx = p_xy / p_x.reshape(-1,1)

print("p(x):")
print(p_x)
print("p(y):")
print(p_y)
print("p(x|y):")
print(p_x_giveny)
print("p(y|x):")
print(p_y_givenx)

p(x):
[0.6 0.4]
p(y):
[0.4 0.5 0.1]
p(x|y):
[[0.75 0.6  0.  ]
 [0.25 0.4  1.  ]]
p(y|x):
[[0.5  0.5  0.  ]
 [0.25 0.5  0.25]]


In [10]:
X_bar = (X * p_x).sum()
Y_bar = (Y * p_y).sum()
X_bar_giveny = np.matmul(X, p_x_giveny)
Y_bar_givenx = np.matmul(p_y_givenx, Y)

cov = 0.0
for i in range(2):
    for j in range(3):
        cov += (X[i]-X_bar)*(Y[j]-Y_bar)*p_xy[i,j]

print("<X>:", X_bar)
print("<Y>:", Y_bar)
print("<X>_p(x|y):", X_bar_giveny)
print("<Y>_p(y|x):", Y_bar_givenx)
print("Cov:", cov)

<X>: 1.4
<Y>: 0.09999999999999998
<X>_p(x|y): [1.25 1.4  2.  ]
<Y>_p(y|x): [-0.5  1. ]
Cov: 0.36000000000000004


### 2. Joint entropy

In [11]:
H_xy = -(p_xy * np.log(p_xy+1e-48)).sum()
print("H[x,y] = %.4f nats" % (H_xy))

H[x,y] = 1.5048 nats


### 3. Marginal entropies

In [12]:
H_x = -(p_x * np.log(p_x)).sum()
H_y = -(p_y * np.log(p_y)).sum()
print("H[x] = %.4f nats" % (H_x))
print("H[y] = %.4f nats" % (H_y))

H[x] = 0.6730 nats
H[y] = 0.9433 nats


### 4. Conditional entropies

In [13]:
H_x_giveny = -(p_xy * np.log(p_x_giveny + 1e-48)).sum()
H_y_givenx = -(p_xy * np.log(p_y_givenx + 1e-48)).sum()
print("H[x|y] = %.4f nats" % (H_x_giveny))
print("H[y|x] = %.4f nats" % (H_y_givenx))

H[x|y] = 0.5614 nats
H[y|x] = 0.8318 nats


### 5. Mutual information

In [14]:
D_kl = (p_xy * (np.log(p_xy+1e-48)-np.log(np.matmul(p_x.reshape(-1,1),p_y.reshape(1,-1))+1e-48))).sum()
I_xy = H_x - H_x_giveny

In [15]:
print("D_kl[p(x,y)||p(x)p(y)] = %.4f" % D_kl)
print("I[x,y] = %.4f" % I_xy)

D_kl[p(x,y)||p(x)p(y)] = 0.1116
I[x,y] = 0.1116


### 7. Verification

$$H[x,y] \leq H[x] + H[y]$$

In [16]:
print("H[x,y] = %.4f" % H_xy)
print("H[x] + H[y] = %.4f" % (H_x + H_y))

H[x,y] = 1.5048
H[x] + H[y] = 1.6164


$$H[x,y] = H[y|x] + H[x]$$

In [17]:
print("H[y|x] + H[x] = %.4f" % (H_y_givenx + H_x))
print("H[x|y] + H[y] = %.4f" % (H_x_giveny + H_y))

H[y|x] + H[x] = 1.5048
H[x|y] + H[y] = 1.5048


$$D_{kl}(x||y) = I[x,y] = H[y] - H[y|x]$$

In [18]:
print("H[y]-H[y|x] = %.4f" % (H_y - H_y_givenx))
print("H[x]-H[x|y] = %.4f" % (H_x - H_x_giveny))
print("D_kl(x||y) = %.4f" % D_kl)
print("I[x,y] = %.4f" % I_xy)

H[y]-H[y|x] = 0.1116
H[x]-H[x|y] = 0.1116
D_kl(x||y) = 0.1116
I[x,y] = 0.1116
